In [1]:
import os
import sys
sys.path
sys.path.append('../../L1DeepMETv2/')
import time
import os.path as osp
import numpy as np
import json
import torch
from torch.autograd import Variable
from tqdm import tqdm

import utils
import model.net as net
import model.data_loader as data_loader
from graphmetnetwork import GraphMetNetwork

import matplotlib.pyplot as plt

/nethome/dvsm3/.conda/envs/deepmet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
n_features_cont = 6
n_features_cat = 2
scale_momentum = 128

In [3]:
def evaluate(model, loss_fn, dataloader, metrics, model_dir, n_features_cont = 6, save_METarr = True, removePuppi = False):
    """Evaluate the model on `num_steps` batches.

    Args:
        model: (torch.nn.Module) the neural network
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches data
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # summary for current eval loop
    loss_avg_arr = []
    qT_arr = []
    
    MET_arr = {
        'genMETx': [],
        'genMETy': [],
        
        'METx': [],
        'METy': [],
        
        'puppiMETx': [],
        'puppiMETy': []
    }
    
    resolutions_arr = {
        'MET':      [[],[],[]],
        'puppiMET': [[],[],[]],
    }

    colors = {
    #    'pfMET': 'black',
        'puppiMET': 'red',
    #    'deepMETResponse': 'blue',
    #    'deepMETResolution': 'green',
        'MET':  'magenta',
    }

    labels = {
    #    'pfMET': 'PF MET',
        'puppiMET': 'PUPPI MET',
    #    'deepMETResponse': 'DeepMETResponse',
    #    'deepMETResolution': 'DeepMETResolution',
        'MET': 'DeepMETv2'
    }

    weights_pdgId_arr = {
        'down': [],
        'up': [],
        'electron': [],
        'muon': [],
        'photon': [],
        'kaon': [],
        'pion': [],
    }
    
    puppi_weights_pdgId_arr = {
        'down': [],
        'up': [],
        'electron': [],
        'muon': [],
        'photon': [],
        'kaon': [],
        'pion': [],
    }
    
    # compute metrics over the dataset
    for data in tqdm(dataloader, desc="Testing Progress", leave=False):
        
        if removePuppi:
            x_cont = data.x[:,:(n_features_cont-1)]
        else:
            x_cont = data.x[:,:n_features_cont]
        
        x_cat = data.x[:,n_features_cont:].long()
        
        # Convert inputs to numpy arrays
        c_x_cont = np.ascontiguousarray(x_cont.squeeze(0).numpy())
        c_x_cat = np.ascontiguousarray(x_cat.squeeze(0).numpy())
        c_batch = np.ascontiguousarray(data.batch.squeeze(0).numpy())
        num_nodes = x_cont.shape[0]
        
        # Run forward method
        model.GraphMetNetworkLayers(c_x_cont, c_x_cat, c_batch, num_nodes)
        
        # Get result
        result = torch.from_numpy(model.get_output())

        loss = loss_fn(result, data.x, data.y, data.batch)

        # compute all metrics on this batch
        resolutions, METs, weights_pdgId, puppi_weights_pdgId = metrics['resolution'](result, data.x, data.y, data.batch, scale_momentum)
        
        for key in resolutions_arr.keys():
            for i in range(len(resolutions_arr[key])):
                resolutions_arr[key][i]=np.concatenate((resolutions_arr[key][i],resolutions[key][i]))

        for key in MET_arr.keys():
            MET_arr[key]=np.concatenate((MET_arr[key],METs[key]))
            
        for pdg in weights_pdgId_arr.keys():
            weights_pdgId_arr[pdg] = np.concatenate((weights_pdgId_arr[pdg],weights_pdgId[pdg]))
            puppi_weights_pdgId_arr[pdg] = np.concatenate((puppi_weights_pdgId_arr[pdg],puppi_weights_pdgId[pdg]))
            
        qT_arr = np.concatenate((qT_arr, METs['genMET']))
        
        loss_avg_arr.append(loss.item())
    
    print('Done Testing, saving results...')
    if save_METarr:
        for key in MET_arr.keys():
            np.savetxt(f'{model_dir}/epoch_{key}.txt', MET_arr[key].ravel(), delimiter = ',')
        for pdg in weights_pdgId_arr.keys():
            np.savetxt(f'{model_dir}/epoch_{pdg}_weights.txt', weights_pdgId_arr[pdg].ravel(), delimiter = ',')
    
    # compute mean of all metrics in summary
    max_x=400 # max qT value
    x_n=20 # number of bins

    bin_edges=np.arange(0, max_x, max_x/x_n)
    
    inds=np.digitize(qT_arr, bin_edges)

    qT_hist=[]
    for i in range(1, len(bin_edges)):
        qT_hist.append((bin_edges[i]+bin_edges[i-1])/2.)
    
    resolution_hists={}
    for key in resolutions_arr:

        R_arr=resolutions_arr[key][2] 
        u_perp_arr=resolutions_arr[key][0]
        u_par_arr=resolutions_arr[key][1]

        u_perp_hist=[]
        u_perp_scaled_hist=[]
        u_par_hist=[]
        u_par_scaled_hist=[]
        R_hist=[]

        for i in range(1, len(bin_edges)):
            R_i=abs(R_arr[np.where(inds==i)[0]])
            R_hist.append(np.mean(R_i))
            
            u_perp_i=u_perp_arr[np.where(inds==i)[0]]
            u_perp_scaled_i=u_perp_i/np.mean(R_i)
            u_perp_hist.append((np.quantile(u_perp_i,0.84)-np.quantile(u_perp_i,0.16))/2.)
            u_perp_scaled_hist.append((np.quantile(u_perp_scaled_i,0.84)-np.quantile(u_perp_scaled_i,0.16))/2.)
            
            u_par_i=u_par_arr[np.where(inds==i)[0]]
            u_par_scaled_i=u_par_i/np.mean(R_i)
            u_par_hist.append((np.quantile(u_par_i,0.84)-np.quantile(u_par_i,0.16))/2.)
            u_par_scaled_hist.append((np.quantile(u_par_scaled_i,0.84)-np.quantile(u_par_scaled_i,0.16))/2.)

        u_perp_resolution=np.histogram(qT_hist, bins=x_n, range=(0,max_x), weights=u_perp_hist)
        u_perp_scaled_resolution=np.histogram(qT_hist, bins=x_n, range=(0,max_x), weights=u_perp_scaled_hist)
        u_par_resolution=np.histogram(qT_hist, bins=x_n, range=(0,max_x), weights=u_par_hist)
        u_par_scaled_resolution=np.histogram(qT_hist, bins=x_n, range=(0,max_x), weights=u_par_scaled_hist)
        R=np.histogram(qT_hist, bins=x_n, range=(0,max_x), weights=R_hist)
        resolution_hists[key] = {
            'u_perp_resolution': u_perp_resolution,
            'u_perp_scaled_resolution': u_perp_scaled_resolution,
            'u_par_resolution': u_par_resolution,
            'u_par_scaled_resolution':u_par_scaled_resolution,
            'R': R
        }
    
    metrics_mean = {
        'loss': np.mean(loss_avg_arr),
        #'resolution': (np.quantile(resolutions_arr,0.84)-np.quantile(resolutions_arr,0.16))/2.
    }
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v)
                                for k, v in metrics_mean.items())
    print("- Eval metrics : " + metrics_string)
    
    return metrics_mean, resolution_hists, MET_arr

In [4]:
data_dir = '../../L1DeepMETv2/data_ttbar'
output_dir = "weights_files/"
dataloaders = data_loader.fetch_dataloader(data_dir = data_dir, batch_size=1, validation_split=.2)
test_dl = dataloaders['test']
loss_fn = net.loss_fn
metrics = net.metrics
model_dir = 'ckpts'

Processing...
0it [00:00, ?it/s]
Done!


Split: 199708
length of train/val data:  798834 199708


/nethome/dvsm3/.conda/envs/deepmet/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [5]:
prefix = '../../L1DeepMETv2/ckpts_April30_scale_sigmoid'
restore_ckpt = osp.join(prefix, 'best.pth.tar')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
norm = torch.tensor([1., 1., 1., 1., 1., 1.]).to(device=device)
torch_model = net.Net(continuous_dim=6, categorical_dim=2 , norm=norm).to(device)
torch_model.eval()
print(torch_model)

/nethome/dvsm3/.conda/envs/deepmet/lib/python3.10/site-packages/torch/cuda/__init__.py:146: UserWarning: 
NVIDIA RTX A6000 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA RTX A6000 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


Net(
  (graphnet): GraphMETNetwork(
    (embed_charge): Embedding(3, 8)
    (embed_pdgid): Embedding(7, 8)
    (embed_continuous): Sequential(
      (0): Linear(in_features=6, out_features=16, bias=True)
      (1): ELU(alpha=1.0)
    )
    (embed_categorical): Sequential(
      (0): Linear(in_features=16, out_features=16, bias=True)
      (1): ELU(alpha=1.0)
    )
    (encode_all): Sequential(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): ELU(alpha=1.0)
    )
    (bn_all): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_continuous): ModuleList(
      (0): ModuleList(
        (0): EdgeConv(nn=Sequential(
          (0): Linear(in_features=64, out_features=32, bias=True)
        ))
        (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ModuleList(
        (0): EdgeConv(nn=Sequential(
          (0): Linear(in_features=64, out_features=32, bias=True)
        ))
     

/nethome/dvsm3/.conda/envs/deepmet/lib/python3.10/site-packages/torch_geometric/nn/conv/message_passing.py:988: UserWarning: 'EdgeConv.jittable' is deprecated and a no-op. Please remove its usage.
  warnings.warn(f"'{self.__class__.__name__}.jittable' is deprecated "


In [6]:
param_restored_new = utils.load_checkpoint(restore_ckpt, torch_model)
weights_dict = param_restored_new['state_dict']
print(weights_dict)

OrderedDict([('graphnet.embed_charge.weight', tensor([[ 0.9159, -1.0585, -0.9913,  0.0120, -1.2546,  0.3010, -0.3681, -0.0259],
        [ 1.1695, -1.1930, -0.4265, -0.7048, -0.3580,  0.5365,  1.0058, -0.7788],
        [-1.1899, -0.7678,  0.9431,  0.2916, -0.2275, -1.3641,  0.6981,  0.4962]],
       device='cuda:0')), ('graphnet.embed_pdgid.weight', tensor([[-2.0383, -0.3847, -0.2413,  0.9122, -0.4805,  0.2302, -0.4746,  0.9317],
        [-0.4117,  1.3556,  0.8550,  0.9977, -0.1622,  1.0582, -0.3220, -0.1858],
        [ 0.3337, -1.7242,  0.2477,  0.0240, -1.0831, -0.7611, -0.2512, -1.4247],
        [-0.0926,  0.4529, -0.2330,  1.5909, -0.3757, -1.6114,  0.4283,  0.3516],
        [-0.8124, -1.1400, -0.2373, -0.4228,  0.0110, -1.9789, -0.4648, -1.3265],
        [ 0.1621,  0.1170, -0.4399,  0.8137, -1.5199,  2.1545,  1.1113, -1.5125],
        [-2.0251,  0.8900,  1.1300, -1.7736, -1.0406, -1.3036, -0.5597, -0.0127]],
       device='cuda:0')), ('graphnet.embed_continuous.0.weight', tensor([[

In [7]:
import shutil

# Check if the directory exists
if os.path.exists(output_dir):
    # Iterate over all the files in the directory
    for filename in os.listdir(output_dir):
        file_path = os.path.join(output_dir, filename)
        try:
            # Check if it's a file and delete it
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            # If it's a directory, delete the directory and its contents
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")
else:
    print(f"Directory {output_dir} does not exist.")


# Function to save the weights as binary files
def save_weights_as_binary(weights_dict, output_dir):
    for key, tensor in weights_dict.items():
        # Convert the tensor to a NumPy array
        np_array = tensor.cpu().numpy()

        # Create a binary file name based on the tensor name
        file_name = output_dir + key.replace('.', '_') + '.bin'

        # Save the NumPy array as a binary file
        np_array.tofile(file_name)
        
# Save all weights in the OrderedDict to binary files
save_weights_as_binary(weights_dict, output_dir)

In [8]:
# Create an instance of the C++ GraphMetNetwork model
model = GraphMetNetwork()

# Load the weights
model.load_weights(output_dir)

In [ ]:
test_metrics = evaluate(model, loss_fn, test_dl, metrics, model_dir)   
resolutions = test_metrics[1]

In [ ]:
# Save metrics in a json file in the model directory
utils.save_dict_to_json(test_metrics, osp.join(model_dir, 'metrics_val_best.json'))
utils.save(resolutions, osp.join(model_dir, 'best.resolutions'))